In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import nltk
from nltk.corpus import stopwords
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('data_combined.csv')
data

,Body ID,Headline,Stance,Headline1,Headline2,articleBody,articleBody1,articleBody2
0,1,"['российский', 'бюджет', 'марте', 'недополучил...",true,российский бюджет в марте недополучил более 30...,российский бюджет март недополучать миллиард р...,Разница между ожидаемыми по итогам марта нефте...,разница ожидаемыми итогам марта нефтегазовыми ...,разница ожидать итог март нефтегазовый доход ф...
1,2,"['банк', 'россии', 'решил', 'снизить', 'ключев...",true,банк россии решил снизить ключевую ставку с 20...,банк россия решать снижать ключевой ставка,Совет директоров Банка России решил снизить кл...,совет директоров банка россии решил снизить кл...,совет директор банк россия решать снижать ключ...
2,3,"['мыс', 'идокопас', 'нато', 'назвали', 'первую...",fake,мыс идокопас в нато назвали первую цель для н...,мыс идокопас нато называть первый цель начало ...,В НАТО составили наступательный план действий ...,нато составили наступательный план действий за...,нато составлять наступательный план действие з...
3,4,"['украине', 'прошли', 'празднования', '300', '...",fake,на украине прошли празднования 300 летия побед...,украина проходить празднование летие победа ка...,Президент Владимир Зеленский принял в Полтаве ...,президент владимир зеленский принял полтаве па...,президент владимир зеленский принимать полтава...
4,5,"['минобороны', 'рф', 'заявило', 'ударах', 'рак...",true,минобороны рф заявило об ударах ракетами «кали...,минобороны заявлять удар ракета калибр запорож...,"Минобороны России объявило, что ракетами больш...",минобороны россии объявило ракетами большой да...,минобороны россия объявлять ракета большой дал...
...,...,...,...,...,...,...,...,...
4403,4404,"['чиновница', 'призвавшая', '«найти', 'покарат...",fake,чиновница призвавшая «найти и покарать предате...,чиновница призывать находить покарать предател...,Заместитель министра просвещения Ирина Карепин...,заместитель министра просвещения ирина карепин...,заместитель министр просвещение ирина карепин ...
4404,4405,"['«включим', 'программу', 'киселёва', 'покажем...",fake,«включим программу киселёва и покажем телевизо...,включать программа киселев показывать телевизо...,Глава штабов Навального Леонид Волков призвал ...,глава штабов навального леонид волков призвал ...,глава штаб навальный леонид волк призывать сво...
4405,4406,"['китайские', 'власти', 'могут', 'казнить', 'з...",fake,китайские власти могут казнить задержанного по...,китайский власть мочь казнить задержанный подо...,Один из менеджеров стройки космодрома Восточны...,менеджеров стройки космодрома восточный взят с...,менеджер стройка космодром восточный взять стр...
4406,4407,"['луганской', 'области', 'переполнены', 'морги...",true,в луганской области переполнены морги хранить...,луганский область переполнять морг хранить тел...,Морги Луганской области переполнены телами пог...,морги луганской области переполнены телами пог...,морг луганский область переполнять тело погибш...


In [3]:
data_test = pd.read_csv('data_test_combined.csv')
data_test

,Body ID,label,Headline,Headline1,Headline2,articleBody,articleBody1,articleBody2
0,1,0,"['лукашенко', 'пригрозил', 'литовским', 'танка...",лукашенко пригрозил литовским танкам белорусск...,лукашенко пригрозить литовский танк белорусски...,"Президент Беларуси Александр Лукашенко заявил,...",президент беларуси александр лукашенко заявил ...,президент беларусь александр лукашенко заявлят...
1,2,1,"['российские', 'компании', 'оказались', 'опасн...",российские компании оказались в опасности из з...,российский компания оказываться опасность глоб...,Несколько крупнейших российских компаний оказа...,несколько крупнейших российских компаний оказа...,несколько крупный российский компания оказыват...
2,3,0,"['лукашенко', 'объявил', 'отмене', 'выборов', ...",лукашенко объявил об отмене выборов из за напа...,лукашенко объявлять отмена выборы нападение ин...,Президент Белоруссии Александр Лукашенко объяв...,президент белоруссии александр лукашенко объяв...,президент белоруссия александр лукашенко объяв...
3,4,1,"['«роснефть»', 'стала', 'лидером', 'объему', '...",«роснефть» стала лидером по объему биржевых пр...,роснефть становиться лидер объем биржевой прод...,С начала июля нефтяные компании заметно увелич...,начала июля нефтяные компании заметно увеличил...,начинать июль нефтяной компания заметно увелич...
4,5,1,"['псковской', 'области', 'дадут', 'десятки', '...",псковской области дадут десятки миллионов рубл...,псковский область давать десяток миллион рубль...,Псковская область получит 189 миллионов рублей...,псковская область получит 189 миллионов рублей...,псковский область получать миллион рубль ремон...
...,...,...,...,...,...,...,...,...
1096,1097,0,"['борис', 'джонсон', 'пообещал', 'избирателям'...",борис джонсон пообещал избирателям вернуть ирл...,борис джонсон пообещать избиратель вернуть ирл...,Кандидат в премьер-министры Великобритании Бор...,кандидат премьер министры великобритании борис...,кандидат премьер министр великобритания борис ...
1097,1098,0,"['михаил', 'горбачев', 'заявил', 'подавал', 'о...",михаил горбачев заявил что не подавал в отстав...,михаил горбачев заявлять подавать отставка пос...,Сенсационное заявление сделал 8 декабря в эфир...,сенсационное заявление сделал 8 декабря эфире ...,сенсационный заявление сделать декабрь эфир не...
1098,1099,0,"['мгимо', 'появится', 'факультет', 'международ...",в мгимо появится факультет международных отнош...,мгимо появляться факультет международный отнош...,Московский государственный институт международ...,московский государственный институт международ...,московский государственный институт международ...
1099,1100,0,"['россии', 'введут', 'штраф', 'отрицание', 'аг...",в россии введут штраф за отрицание агрессии нато,россия вводить штраф отрицание агрессия нато,На рассмотрение в Государственную думу поступи...,рассмотрение государственную думу поступил зак...,рассмотрение государственный дума поступать за...


In [4]:
data_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1101 entries, 0 to 1100
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Body ID       1101 non-null   int64 
 1   label         1101 non-null   int64 
 2   Headline      1101 non-null   object
 3   Headline1     1101 non-null   object
 4   Headline2     1101 non-null   object
 5   articleBody   1101 non-null   object
 6   articleBody1  1101 non-null   object
 7   articleBody2  1101 non-null   object
dtypes: int64(2), object(6)
memory usage: 68.9+ KB


In [5]:
data['Stance'] = data['Stance'].replace(['true', 'fake'], [1, 0]).astype('int')

In [6]:
data

,Body ID,Headline,Stance,Headline1,Headline2,articleBody,articleBody1,articleBody2
0,1,"['российский', 'бюджет', 'марте', 'недополучил...",1,российский бюджет в марте недополучил более 30...,российский бюджет март недополучать миллиард р...,Разница между ожидаемыми по итогам марта нефте...,разница ожидаемыми итогам марта нефтегазовыми ...,разница ожидать итог март нефтегазовый доход ф...
1,2,"['банк', 'россии', 'решил', 'снизить', 'ключев...",1,банк россии решил снизить ключевую ставку с 20...,банк россия решать снижать ключевой ставка,Совет директоров Банка России решил снизить кл...,совет директоров банка россии решил снизить кл...,совет директор банк россия решать снижать ключ...
2,3,"['мыс', 'идокопас', 'нато', 'назвали', 'первую...",0,мыс идокопас в нато назвали первую цель для н...,мыс идокопас нато называть первый цель начало ...,В НАТО составили наступательный план действий ...,нато составили наступательный план действий за...,нато составлять наступательный план действие з...
3,4,"['украине', 'прошли', 'празднования', '300', '...",0,на украине прошли празднования 300 летия побед...,украина проходить празднование летие победа ка...,Президент Владимир Зеленский принял в Полтаве ...,президент владимир зеленский принял полтаве па...,президент владимир зеленский принимать полтава...
4,5,"['минобороны', 'рф', 'заявило', 'ударах', 'рак...",1,минобороны рф заявило об ударах ракетами «кали...,минобороны заявлять удар ракета калибр запорож...,"Минобороны России объявило, что ракетами больш...",минобороны россии объявило ракетами большой да...,минобороны россия объявлять ракета большой дал...
...,...,...,...,...,...,...,...,...
4403,4404,"['чиновница', 'призвавшая', '«найти', 'покарат...",0,чиновница призвавшая «найти и покарать предате...,чиновница призывать находить покарать предател...,Заместитель министра просвещения Ирина Карепин...,заместитель министра просвещения ирина карепин...,заместитель министр просвещение ирина карепин ...
4404,4405,"['«включим', 'программу', 'киселёва', 'покажем...",0,«включим программу киселёва и покажем телевизо...,включать программа киселев показывать телевизо...,Глава штабов Навального Леонид Волков призвал ...,глава штабов навального леонид волков призвал ...,глава штаб навальный леонид волк призывать сво...
4405,4406,"['китайские', 'власти', 'могут', 'казнить', 'з...",0,китайские власти могут казнить задержанного по...,китайский власть мочь казнить задержанный подо...,Один из менеджеров стройки космодрома Восточны...,менеджеров стройки космодрома восточный взят с...,менеджер стройка космодром восточный взять стр...
4406,4407,"['луганской', 'области', 'переполнены', 'морги...",1,в луганской области переполнены морги хранить...,луганский область переполнять морг хранить тел...,Морги Луганской области переполнены телами пог...,морги луганской области переполнены телами пог...,морг луганский область переполнять тело погибш...


In [7]:
data = data.set_index("Body ID")
data

,Headline,Stance,Headline1,Headline2,articleBody,articleBody1,articleBody2
Body ID,,,,,,,
1,"['российский', 'бюджет', 'марте', 'недополучил...",1,российский бюджет в марте недополучил более 30...,российский бюджет март недополучать миллиард р...,Разница между ожидаемыми по итогам марта нефте...,разница ожидаемыми итогам марта нефтегазовыми ...,разница ожидать итог март нефтегазовый доход ф...
2,"['банк', 'россии', 'решил', 'снизить', 'ключев...",1,банк россии решил снизить ключевую ставку с 20...,банк россия решать снижать ключевой ставка,Совет директоров Банка России решил снизить кл...,совет директоров банка россии решил снизить кл...,совет директор банк россия решать снижать ключ...
3,"['мыс', 'идокопас', 'нато', 'назвали', 'первую...",0,мыс идокопас в нато назвали первую цель для н...,мыс идокопас нато называть первый цель начало ...,В НАТО составили наступательный план действий ...,нато составили наступательный план действий за...,нато составлять наступательный план действие з...
4,"['украине', 'прошли', 'празднования', '300', '...",0,на украине прошли празднования 300 летия побед...,украина проходить празднование летие победа ка...,Президент Владимир Зеленский принял в Полтаве ...,президент владимир зеленский принял полтаве па...,президент владимир зеленский принимать полтава...
5,"['минобороны', 'рф', 'заявило', 'ударах', 'рак...",1,минобороны рф заявило об ударах ракетами «кали...,минобороны заявлять удар ракета калибр запорож...,"Минобороны России объявило, что ракетами больш...",минобороны россии объявило ракетами большой да...,минобороны россия объявлять ракета большой дал...
...,...,...,...,...,...,...,...
4404,"['чиновница', 'призвавшая', '«найти', 'покарат...",0,чиновница призвавшая «найти и покарать предате...,чиновница призывать находить покарать предател...,Заместитель министра просвещения Ирина Карепин...,заместитель министра просвещения ирина карепин...,заместитель министр просвещение ирина карепин ...
4405,"['«включим', 'программу', 'киселёва', 'покажем...",0,«включим программу киселёва и покажем телевизо...,включать программа киселев показывать телевизо...,Глава штабов Навального Леонид Волков призвал ...,глава штабов навального леонид волков призвал ...,глава штаб навальный леонид волк призывать сво...
4406,"['китайские', 'власти', 'могут', 'казнить', 'з...",0,китайские власти могут казнить задержанного по...,китайский власть мочь казнить задержанный подо...,Один из менеджеров стройки космодрома Восточны...,менеджеров стройки космодрома восточный взят с...,менеджер стройка космодром восточный взять стр...


In [8]:
data['Headline'] = data['Headline'].apply(lambda x: "".join(x.replace("[", "").replace("]", "").replace("'", "").split(',')))
data

,Headline,Stance,Headline1,Headline2,articleBody,articleBody1,articleBody2
Body ID,,,,,,,
1,российский бюджет марте недополучил 300 миллиа...,1,российский бюджет в марте недополучил более 30...,российский бюджет март недополучать миллиард р...,Разница между ожидаемыми по итогам марта нефте...,разница ожидаемыми итогам марта нефтегазовыми ...,разница ожидать итог март нефтегазовый доход ф...
2,банк россии решил снизить ключевую ставку 20 17,1,банк россии решил снизить ключевую ставку с 20...,банк россия решать снижать ключевой ставка,Совет директоров Банка России решил снизить кл...,совет директоров банка россии решил снизить кл...,совет директор банк россия решать снижать ключ...
3,мыс идокопас нато назвали первую цель начала п...,0,мыс идокопас в нато назвали первую цель для н...,мыс идокопас нато называть первый цель начало ...,В НАТО составили наступательный план действий ...,нато составили наступательный план действий за...,нато составлять наступательный план действие з...
4,украине прошли празднования 300 летия победы к...,0,на украине прошли празднования 300 летия побед...,украина проходить празднование летие победа ка...,Президент Владимир Зеленский принял в Полтаве ...,президент владимир зеленский принял полтаве па...,президент владимир зеленский принимать полтава...
5,минобороны рф заявило ударах ракетами «калибр»...,1,минобороны рф заявило об ударах ракетами «кали...,минобороны заявлять удар ракета калибр запорож...,"Минобороны России объявило, что ракетами больш...",минобороны россии объявило ракетами большой да...,минобороны россия объявлять ракета большой дал...
...,...,...,...,...,...,...,...
4404,чиновница призвавшая «найти покарать предателе...,0,чиновница призвавшая «найти и покарать предате...,чиновница призывать находить покарать предател...,Заместитель министра просвещения Ирина Карепин...,заместитель министра просвещения ирина карепин...,заместитель министр просвещение ирина карепин ...
4405,«включим программу киселёва покажем телевизора...,0,«включим программу киселёва и покажем телевизо...,включать программа киселев показывать телевизо...,Глава штабов Навального Леонид Волков призвал ...,глава штабов навального леонид волков призвал ...,глава штаб навальный леонид волк призывать сво...
4406,китайские власти могут казнить задержанного по...,0,китайские власти могут казнить задержанного по...,китайский власть мочь казнить задержанный подо...,Один из менеджеров стройки космодрома Восточны...,менеджеров стройки космодрома восточный взят с...,менеджер стройка космодром восточный взять стр...


In [9]:
data_test['Stance'] = data_test['label']
data_test.drop('label', axis=1, inplace=True)
data_test

,Body ID,Headline,Headline1,Headline2,articleBody,articleBody1,articleBody2,Stance
0,1,"['лукашенко', 'пригрозил', 'литовским', 'танка...",лукашенко пригрозил литовским танкам белорусск...,лукашенко пригрозить литовский танк белорусски...,"Президент Беларуси Александр Лукашенко заявил,...",президент беларуси александр лукашенко заявил ...,президент беларусь александр лукашенко заявлят...,0
1,2,"['российские', 'компании', 'оказались', 'опасн...",российские компании оказались в опасности из з...,российский компания оказываться опасность глоб...,Несколько крупнейших российских компаний оказа...,несколько крупнейших российских компаний оказа...,несколько крупный российский компания оказыват...,1
2,3,"['лукашенко', 'объявил', 'отмене', 'выборов', ...",лукашенко объявил об отмене выборов из за напа...,лукашенко объявлять отмена выборы нападение ин...,Президент Белоруссии Александр Лукашенко объяв...,президент белоруссии александр лукашенко объяв...,президент белоруссия александр лукашенко объяв...,0
3,4,"['«роснефть»', 'стала', 'лидером', 'объему', '...",«роснефть» стала лидером по объему биржевых пр...,роснефть становиться лидер объем биржевой прод...,С начала июля нефтяные компании заметно увелич...,начала июля нефтяные компании заметно увеличил...,начинать июль нефтяной компания заметно увелич...,1
4,5,"['псковской', 'области', 'дадут', 'десятки', '...",псковской области дадут десятки миллионов рубл...,псковский область давать десяток миллион рубль...,Псковская область получит 189 миллионов рублей...,псковская область получит 189 миллионов рублей...,псковский область получать миллион рубль ремон...,1
...,...,...,...,...,...,...,...,...
1096,1097,"['борис', 'джонсон', 'пообещал', 'избирателям'...",борис джонсон пообещал избирателям вернуть ирл...,борис джонсон пообещать избиратель вернуть ирл...,Кандидат в премьер-министры Великобритании Бор...,кандидат премьер министры великобритании борис...,кандидат премьер министр великобритания борис ...,0
1097,1098,"['михаил', 'горбачев', 'заявил', 'подавал', 'о...",михаил горбачев заявил что не подавал в отстав...,михаил горбачев заявлять подавать отставка пос...,Сенсационное заявление сделал 8 декабря в эфир...,сенсационное заявление сделал 8 декабря эфире ...,сенсационный заявление сделать декабрь эфир не...,0
1098,1099,"['мгимо', 'появится', 'факультет', 'международ...",в мгимо появится факультет международных отнош...,мгимо появляться факультет международный отнош...,Московский государственный институт международ...,московский государственный институт международ...,московский государственный институт международ...,0
1099,1100,"['россии', 'введут', 'штраф', 'отрицание', 'аг...",в россии введут штраф за отрицание агрессии нато,россия вводить штраф отрицание агрессия нато,На рассмотрение в Государственную думу поступи...,рассмотрение государственную думу поступил зак...,рассмотрение государственный дума поступать за...,0


In [10]:
data_test = data_test.set_index("Body ID")
data_test

,Headline,Headline1,Headline2,articleBody,articleBody1,articleBody2,Stance
Body ID,,,,,,,
1,"['лукашенко', 'пригрозил', 'литовским', 'танка...",лукашенко пригрозил литовским танкам белорусск...,лукашенко пригрозить литовский танк белорусски...,"Президент Беларуси Александр Лукашенко заявил,...",президент беларуси александр лукашенко заявил ...,президент беларусь александр лукашенко заявлят...,0
2,"['российские', 'компании', 'оказались', 'опасн...",российские компании оказались в опасности из з...,российский компания оказываться опасность глоб...,Несколько крупнейших российских компаний оказа...,несколько крупнейших российских компаний оказа...,несколько крупный российский компания оказыват...,1
3,"['лукашенко', 'объявил', 'отмене', 'выборов', ...",лукашенко объявил об отмене выборов из за напа...,лукашенко объявлять отмена выборы нападение ин...,Президент Белоруссии Александр Лукашенко объяв...,президент белоруссии александр лукашенко объяв...,президент белоруссия александр лукашенко объяв...,0
4,"['«роснефть»', 'стала', 'лидером', 'объему', '...",«роснефть» стала лидером по объему биржевых пр...,роснефть становиться лидер объем биржевой прод...,С начала июля нефтяные компании заметно увелич...,начала июля нефтяные компании заметно увеличил...,начинать июль нефтяной компания заметно увелич...,1
5,"['псковской', 'области', 'дадут', 'десятки', '...",псковской области дадут десятки миллионов рубл...,псковский область давать десяток миллион рубль...,Псковская область получит 189 миллионов рублей...,псковская область получит 189 миллионов рублей...,псковский область получать миллион рубль ремон...,1
...,...,...,...,...,...,...,...
1097,"['борис', 'джонсон', 'пообещал', 'избирателям'...",борис джонсон пообещал избирателям вернуть ирл...,борис джонсон пообещать избиратель вернуть ирл...,Кандидат в премьер-министры Великобритании Бор...,кандидат премьер министры великобритании борис...,кандидат премьер министр великобритания борис ...,0
1098,"['михаил', 'горбачев', 'заявил', 'подавал', 'о...",михаил горбачев заявил что не подавал в отстав...,михаил горбачев заявлять подавать отставка пос...,Сенсационное заявление сделал 8 декабря в эфир...,сенсационное заявление сделал 8 декабря эфире ...,сенсационный заявление сделать декабрь эфир не...,0
1099,"['мгимо', 'появится', 'факультет', 'международ...",в мгимо появится факультет международных отнош...,мгимо появляться факультет международный отнош...,Московский государственный институт международ...,московский государственный институт международ...,московский государственный институт международ...,0


In [11]:
data_test['Headline'] = data_test['Headline'].apply(lambda x: "".join(x.replace("[", "").replace("]", "").replace("'", "").split(',')))
data_test

,Headline,Headline1,Headline2,articleBody,articleBody1,articleBody2,Stance
Body ID,,,,,,,
1,лукашенко пригрозил литовским танкам белорусск...,лукашенко пригрозил литовским танкам белорусск...,лукашенко пригрозить литовский танк белорусски...,"Президент Беларуси Александр Лукашенко заявил,...",президент беларуси александр лукашенко заявил ...,президент беларусь александр лукашенко заявлят...,0
2,российские компании оказались опасности глобал...,российские компании оказались в опасности из з...,российский компания оказываться опасность глоб...,Несколько крупнейших российских компаний оказа...,несколько крупнейших российских компаний оказа...,несколько крупный российский компания оказыват...,1
3,лукашенко объявил отмене выборов нападения ино...,лукашенко объявил об отмене выборов из за напа...,лукашенко объявлять отмена выборы нападение ин...,Президент Белоруссии Александр Лукашенко объяв...,президент белоруссии александр лукашенко объяв...,президент белоруссия александр лукашенко объяв...,0
4,«роснефть» стала лидером объему биржевых прода...,«роснефть» стала лидером по объему биржевых пр...,роснефть становиться лидер объем биржевой прод...,С начала июля нефтяные компании заметно увелич...,начала июля нефтяные компании заметно увеличил...,начинать июль нефтяной компания заметно увелич...,1
5,псковской области дадут десятки миллионов рубл...,псковской области дадут десятки миллионов рубл...,псковский область давать десяток миллион рубль...,Псковская область получит 189 миллионов рублей...,псковская область получит 189 миллионов рублей...,псковский область получать миллион рубль ремон...,1
...,...,...,...,...,...,...,...
1097,борис джонсон пообещал избирателям вернуть ирл...,борис джонсон пообещал избирателям вернуть ирл...,борис джонсон пообещать избиратель вернуть ирл...,Кандидат в премьер-министры Великобритании Бор...,кандидат премьер министры великобритании борис...,кандидат премьер министр великобритания борис ...,0
1098,михаил горбачев заявил подавал отставку поста ...,михаил горбачев заявил что не подавал в отстав...,михаил горбачев заявлять подавать отставка пос...,Сенсационное заявление сделал 8 декабря в эфир...,сенсационное заявление сделал 8 декабря эфире ...,сенсационный заявление сделать декабрь эфир не...,0
1099,мгимо появится факультет международных отношен...,в мгимо появится факультет международных отнош...,мгимо появляться факультет международный отнош...,Московский государственный институт международ...,московский государственный институт международ...,московский государственный институт международ...,0


In [12]:
nltk.download("stopwords")
russian_stopwords = stopwords.words("russian")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ivan\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [15]:
param_grid = [
  {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
  {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf'], 'probability': [True]},
 ]

clf = GridSearchCV(estimator=SVC(), param_grid=param_grid, cv=3,  n_jobs=-1)

In [16]:
vectorizer = CountVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=russian_stopwords)
vectorizer.fit(data['articleBody1'].values.astype('U'))         
vectorizer.fit(data_test['articleBody1'].values.astype('U'))   
X_train = vectorizer.transform(data['articleBody1'].values.astype('U')).toarray()         
X_test = vectorizer.transform(data_test['articleBody1'].values.astype('U')).toarray()    

In [17]:
%%time
clf.fit(X_train, data.Stance)
svc_best = clf.best_estimator_
svc_best.fit(X_train, data.Stance)
preds = svc_best.predict(X_test)
print(f"Точность на тесте: {round(accuracy_score(data_test.Stance, preds), 3)}")

Точность на тесте: 0.95
CPU times: total: 12.6 s
Wall time: 3min 43s


In [18]:
import joblib

In [19]:
joblib.dump(svc_best, 'best.pkl')
joblib.dump(vectorizer, 'vectorizer.pkl')

['vectorizer.pkl']

In [22]:
import joblib

model = joblib.load('best.pkl')
vectorizer = joblib.load('vectorizer.pkl')

new_text = "какая то новость"
new_text_vectorized = vectorizer.transform([new_text]).toarray()
probabilities = model.predict_proba(new_text_vectorized)

predicted_class = probabilities.argmax()

predicted_probability = probabilities[0][predicted_class]
if predicted_class == 0:
    print("Фейк")
else:
    print("Правда")
print(f"Вероятность: {predicted_probability:.2f}")


Фейк
Вероятность: 0.60
